# DAND - Data Wrangling Project

## Objective

Demonstrate competency wrangling mulitple data sources, principally the "WeRateDogs" twitter account, to produce 'interesting and trustworthy' analyses and visualizations. This requires gathering, assessing, cleaning several sources.

### Requirements

The below constraints were documented in the project requirements:
 - Use only original rating tweets that have images; do not use retweets and non-rating tweets
 - At least (8) quality issues and (2) tidyness issues must be documented and remediated
 - At least (3) insights and (1) visualization must be produced
 - Written reports must be prepared:
     - 300 - 600 words describing the wrangling efforts (named 'wrangle_report'; submitted in PDF or HMTL)
     - 250-word minimum communicating insights and analyses (named 'act_report'; submitted in PDF or HTML)
 - Store the clean DataFrame in a CSV named 'twitter_archive_master.csv', as well as other tables required for tidiness
 - The "WeRateDogs" twitter archive must be downloaded manually and read into the .ipynb
 - The tweet image predictions must be requested programmatically using the provided URL
 - API tokens or credentials must not be included in final submission
 - Wrangling must capture each rating's:
     - Count of retweets
     - Count of favorite/"like" interactions
     - Tweet ID


### Explicit Non-Requirements

The below were explicitly listed as non-requirements:
   - Full sanitization of all data sources
   - Rating ratios > 1 are valid
   - Tweets do not need to be gathered beyond Aug 1 2017

### Mandatory Data Sources

Udacity requires the use of the (3) data sources below:
 - An twitter archive of the "WeRateDogs" account; provided in CSV format
 - Additional data from the Twitter API; using Tweepy
 - Image prediction data from a Udacity neural net; hosted in TSV format

## Methodology
Per Udacity instruction and best practice, the project will flow as such:
 
 ### Data Gathering
     Loading all of the required data sources, ensuring consistent encoding across each.
     
 ### Data Assessment
     Manually and programmatically examine the data sources to identify potential issues relating to:

   #### Quality
        - Completeness : Are there missing records or values within and between tables?
        - Accuracy : Is there wrong data that conforms to each column's validation rules?
        - Consistency : Is the same information represented consistently across sources?
        - Validity : Does any data break validation rules required by our schema? Do they defy real-world constraints?
        
   #### Tidiness
        - Does each column represent a distinct variable?
        - Does each observation occupy its own row?
        - Does each type of observation have its own table?
 
 ### Data Cleaning
     Addressing the identified quality and tidiness issues, where possible and sensible to do so. This should procede following:
         1) Defining exactly the issue to clean and how it should be done
         2) Coding a solution to address the issue
         3) Testing the solution to ensure solution performed as intended
         
Documentation of these tasks and accomplished in-line narrative using markdown cells, as well as in code comments.

# Data Wrangling

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import json
import re
# I'll want to manually review some of the underlying data to get context, so I don't want URLs truncated
pd.set_option('display.max_colwidth', -1)

## Data Gathering

### Twitter Archive

In [2]:
# File was manually uploaded to the home directory per instructions. Called into a new dataframe here.
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv',encoding='utf-8')
twitter_archive.head()
import matplotlib.pyplot as plt

### Twitter API

In [3]:
import tweepy
# Twitter API authentication
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

# Instance the API
api = tweepy.API(auth,
                 wait_on_rate_limit=True,
                 wait_on_rate_limit_notify=True)

In [4]:
# Loop with try / except blocks to pull the tweets from the IDs that exist in the twitter_archive. Writes to a text file
# Failure to try / except will frustrate - the API call will run for a long time and produce errors when NaNs are encountered
with open('tweet_json.txt','w',encoding='utf-8') as file:
    for tweet_id in twitter_archive.tweet_id:
        try:
            tweet = api.get_status(tweet_id)
            json.dump(tweet._json, file)
            file.write('\n')
        except:
            continue

Rate limit reached. Sleeping for: 733
Rate limit reached. Sleeping for: 732


In [5]:
twitter_file = 'tweet_json.txt'
with open(twitter_file,'r') as file:
    line = file.readline()
    tweet = json.loads(line)
    print(tweet) 

{'created_at': 'Tue Aug 01 16:23:56 +0000 2017', 'id': 892420643555336193, 'id_str': '892420643555336193', 'text': "This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU", 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 892420639486877696, 'id_str': '892420639486877696', 'indices': [86, 109], 'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg', 'url': 'https://t.co/MgUWQ76dJU', 'display_url': 'pic.twitter.com/MgUWQ76dJU', 'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'medium': {'w': 540, 'h': 528, 'resize': 'fit'}, 'small': {'w': 540, 'h': 528, 'resize': 'fit'}, 'large': {'w': 540, 'h': 528, 'resize': 'fit'}}}]}, 'extended_entities': {'media': [{'id': 892420639486877696, 'id_s

In [6]:
# Making a new dataframe to append data from the above JSON
twitter_api_data = pd.DataFrame(columns = ['tweet_id','favorite_count','retweet_count'])
with open(twitter_file,'r',encoding='utf-8') as file:
    for line in file:
        tweet = json.loads(line)
        twitter_api_data = twitter_api_data.append({'tweet_id':int(tweet['id']),
                                                    'favorite_count':int(tweet['favorite_count']),
                                                    'retweet_count':int(tweet['retweet_count'])},ignore_index=True)

In [7]:
twitter_api_data.head()

,tweet_id,favorite_count,retweet_count
0,892420643555336193,37853,8259
1,892177421306343426,32526,6103
2,891815181378084864,24490,4041
3,891689557279858688,41206,8410
4,891327558926688256,39388,9108


### Image Predictions

In [8]:
# Defines URL and extracts the file name from the end; uses requests to get the file and encode as utf-8
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
file_name = url.split('/')[-1]
response = requests.get(url)
response.encoding = 'utf-8'
response_content = response.text

# Checks to see if the file exists locally. If not, fetches the file from the URL, opens it, and writes locally
if not os.path.exists(file_name):
        open(file_name,'wb').write(response.content)
        
        # Uses local TSV to create a new dataframe
image_predictions = pd.read_csv(file_name,sep="\t")
image_predictions.name = 'Image Predictions'
image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


##### Explanation of Approach
Though there was an explicit requirement to request the file programmatically, it is also a best practice to improve the integrity and replicability of any analyses. While it would be possible to read the remotely-hosted TSV file directly into a pandas dataframe, illustrated as the "easy_way_images" dataframe below, this would leave the code susceptible to breaking if the pathway were to change. 

Because we do not control the remotely hosted file or its pathway, we opt to make a local copy if one does not already exist. Thus, if the pathway changes, the analysis can still be performed on the original, in-scope data. This approach would be weak if there is an on-going update to the fetched file, as the code above would not copy a new version when a local copy already exists. However, there is no explicit requirement for this analysis to be performed on an on-going basis, consuming iterable updates to the underlying data.
##### 'The Easy Way'

In [9]:
easy_way_images = pd.read_csv('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv',sep="\t")
easy_way_images.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


# Data Assessment

In [10]:
print ("Records in Twitter Archive: ",len(twitter_archive))
print ("Records in Twitter API Data: ",len(twitter_api_data))
print ("Records in Image Predictions: ",len(image_predictions))

Records in Twitter Archive:  2356
Records in Twitter API Data:  2340
Records in Image Predictions:  2075


In [11]:
# Check for duplicate columns across dataframes
all_columns = pd.Series(list(twitter_archive) + list(twitter_api_data) + list(image_predictions))
all_columns[all_columns.duplicated()]

17    tweet_id
20    tweet_id
dtype: object

What good fortune, a common key between all the sources and no extra columns to evaluate. Now let's look at each dataframe's structure.

In [12]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [13]:
twitter_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [14]:
# Check proportion of NaNs in each column
twitter_archive.isnull().sum()/len(twitter_archive)

tweet_id                      0.000000
in_reply_to_status_id         0.966893
in_reply_to_user_id           0.966893
timestamp                     0.000000
source                        0.000000
text                          0.000000
retweeted_status_id           0.923175
retweeted_status_user_id      0.923175
retweeted_status_timestamp    0.923175
expanded_urls                 0.025042
rating_numerator              0.000000
rating_denominator            0.000000
name                          0.000000
doggo                         0.000000
floofer                       0.000000
pupper                        0.000000
puppo                         0.000000
dtype: float64

Since we eventually need to filter not just for unique tweets, but unique rating tweets, let's check for duplicates in some columns that might help us find exclusions

In [15]:
print("Potential Dupes - twitter_archive.tweet_id - ",len(twitter_archive.tweet_id)-len(twitter_archive.tweet_id.drop_duplicates()))
print("Potential Dupes - twitter_archive.expanded_urls - ",len(twitter_archive.expanded_urls)-len(twitter_archive.expanded_urls.drop_duplicates()))
print("Potential Dupes - twitter_archive.text - ",len(twitter_archive.text)-len(twitter_archive.text.drop_duplicates()))

Potential Dupes - twitter_archive.tweet_id -  0
Potential Dupes - twitter_archive.expanded_urls -  137
Potential Dupes - twitter_archive.text -  0


We learn that each row is in-fact a unique tweet, suggesting that the archive has tidy rows. However, there are duplication among the URLs (denoting an image with the tweet - one of the constraints). There may be dirty data here, or worse. Perhaps WeRateDogs is recycling content?!

In [16]:
twitter_api_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 3 columns):
tweet_id          2340 non-null object
favorite_count    2340 non-null object
retweet_count     2340 non-null object
dtypes: object(3)
memory usage: 54.9+ KB


In [17]:
twitter_api_data.describe()

,tweet_id,favorite_count,retweet_count
count,2340,2340,2340
unique,2340,2008,1694
top,667495797102141441,0,1011
freq,1,167,5


In [18]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [19]:
image_predictions.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [20]:
print("Potential Dupes - image_predictions.tweet - ",len(image_predictions.tweet_id)-len(image_predictions.tweet_id.drop_duplicates()))
print("Potential Dupes - image_predictions.jpg_url - ",len(image_predictions.jpg_url)-len(image_predictions.jpg_url.drop_duplicates()))

Potential Dupes - image_predictions.tweet -  0
Potential Dupes - image_predictions.jpg_url -  66


Our duplicate check here confirms tidiness of 'image_predictions' table. Duplcation of the image URLs is somewhat expected, since the neural net probably points to the same photo for every type of prediction it makes. If multiple dogs are predicted to be of the same breed this would be expected, for somewhat obvious example.

##### Examining Columns of Interest

We may be eventually curious about where the account is operating from, so it may be interesting to understand the 'source' column. And from when I printed the dataframe's header, the column looked like a mess that included HTML. How complex is it really, though?

In [21]:
twitter_archive.source.unique()

array([ '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
       '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>',
       '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'], dtype=object)

That's good news. For all the visual mess, there are only (4) unique values, which create neat categories for analysis. The strings just need to be parsed, which is made easy by the reliable placement of brackets in HTML.

Next up, to get a better grip on the columns covering each dog's 'stage':

In [22]:
examine_cols = ['doggo','floofer','pupper','puppo']
for col in examine_cols:
    print(twitter_archive[col].unique())

['None' 'doggo']
['None' 'floofer']
['None' 'pupper']
['None' 'puppo']


These look pretty clean too, though they're pretty useless as strings.

It may be interesting to eventually look at where the images the account posts are hosted, to identify any persistent trends or collaborations. But my eyes won't be happy looking at all the URLs, and their unique pathways don't lend well to aggregation. But maybe we can parse this down to the parent domains and count those.

In [23]:
# Find parent of URL
t = twitter_archive.expanded_urls.str.replace('www.|https://|http://','')
t.head()
t2 = t.str.split('/',n=2,expand=True)
t2[0].value_counts()

#t3 = t2[2].str.split('.',n=1,expand=True)
#t3[0].unique()

twitter.com            2149
vine.co                103 
gofundme.com           33  
us.blastingnews.com    3   
petfinder.com          2   
loveyourmelon.com      2   
weratedogs.com         1   
goo.gl                 1   
m.facebook.com         1   
m.youtube.com          1   
patreon.com            1   
Name: 0, dtype: int64

In [24]:
# Non-twitter or vine image sources
# This will not include rows with multiple URLs if one is to twitter or vine
twitter_archive[twitter_archive['expanded_urls'].str.contains('twitter|vine')==False]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
335,832645525019123713,NaN,NaN,2017-02-17 17:38:57 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",There's going to be a dog terminal at JFK Airport. This is not a drill. 10/10 \nhttps://t.co/dp5h9bCwU7,NaN,NaN,NaN,http://us.blastingnews.com/news/2017/02/jfk-announces-its-first-ever-ark-oasis-animal-terminal-001480161.html?sbdht=_pM1QUzk3wsdTxcmMoRPV7FWYYlsNKcFRcYSY7OmeHnOXA4NtUM6PLQ2_,10,10,not,None,None,None,None
444,819238181065359361,NaN,NaN,2017-01-11 17:42:57 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",Some happy pupper news to share. 10/10 for everyone involved \nhttps://t.co/MefMAZX2uv,NaN,NaN,NaN,http://us.blastingnews.com/news/2017/01/200-dogs-saved-from-south-korean-dog-meat-industry-001385441.html?sbdht=_pM1QUzk3wsfscF9XF2WEd9KoWDpsQlMUjfh1HxxUq0u5mMbiu2B0kw2_,10,10,None,None,None,pupper,None
754,778764940568104960,NaN,NaN,2016-09-22 01:16:45 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Oh my god it's Narcos but Barkos. 13/10 someone please make this happen\nhttps://t.co/tird9cIlzB,NaN,NaN,NaN,https://m.youtube.com/watch?v=idKxCMsS3FQ&feature=youtu.be,13,10,None,None,None,None,None
885,760153949710192640,NaN,NaN,2016-08-01 16:43:19 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","RT @hownottodraw: The story/person behind @dog_rates is heckin adorable af. 11/10, probably would pet. https://t.co/AG5UnRrmzJ",7.601538e+17,195036846.0,2016-08-01 16:42:51 +0000,"https://weratedogs.com/pages/about-us,https://weratedogs.com/pages/about-us",11,10,None,None,None,None,None


Of course, most of the tweets are pointing toward a twitter or vine URL. I printed out some of the ones that don't point to twitter or vine for a cursory examination, and the first item shows some promise - it's not a rating of a dog, but of JFK airport's proposed dog terminal. This line of pursuit may yield more quality issues.

Next I'm curious how to treat the dog's' 'stage' field. The project instructions strongly implied that these could be collapsed into a 'stage' column, with a categorical value for each. But that doesn't feel right either, since the definitions provided for each weren't mutually exclusive. And I have strong aprioristic biases based on my experiences with dogs.

But maybe WeRateDogs or Udacity has done some work in the data that can suggest the best treatment?

In [25]:
# Check to see if overlap between doggo classification columns
print('Both doggo and floofer: ',len(twitter_archive.query('doggo!="None" and floofer!="None"')))
print('Both doggo and pupper: ',len(twitter_archive.query('doggo!="None" and pupper!="None"')))
print('Both doggo and puppo: ',len(twitter_archive.query('doggo!="None" and puppo!="None"')))
print('Both puppo and floofer',len(twitter_archive.query('puppo!="None" and floofer!="None"')))
print('Both puppo and pupper',len(twitter_archive.query('puppo!="None" and pupper!="None"')))
print('Both pupper and floofer',len(twitter_archive.query('pupper!="None" and floofer!="None"')))

Both doggo and floofer:  1
Both doggo and pupper:  12
Both doggo and puppo:  1
Both puppo and floofer 0
Both puppo and pupper 0
Both pupper and floofer 0


With only 14 records occupying at least two states, there is a strong argument that this could be dirty and the result of the methods used to parse the tweets and populate the fields. However, this is also most common with 'doggo' and 'pupper', who's definitions in the Dogtionary are recursive with one another. 

https://d17h27t6h515a5.cloudfront.net/topher/2017/October/59e04ceb_dogtionary-combined/dogtionary-combined.png

I also happen to know, first hand, that many pups exists in a superposition between doggo and pupper, as well as other dog descriptors. 

At a minimum, some of the complexity can be reduced and the data can become more intelligible by replacing the values with booleans or 1's and 0's so that the columns represent the data consistently.

Now to look at the names, which (when parsed correctly) look like proper nouns starting with a capital. Maybe there's some dirty data here we could easily find, just by looking for values that are all lowercase.

In [26]:
names_all_lowercase = twitter_archive.name.str.islower()
print(len(twitter_archive[names_all_lowercase]))

twitter_archive[names_all_lowercase]

109


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
22,887517139158093824,NaN,NaN,2017-07-19 03:39:09 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",I've yet to rate a Venezuelan Hover Wiener. This is such an honor. 14/10 paw-inspiring af (IG: roxy.thedoxy) https://t.co/20VrLAA8ba,NaN,NaN,NaN,https://twitter.com/dog_rates/status/887517139158093824/video/1,14,10,such,None,None,None,None
56,881536004380872706,NaN,NaN,2017-07-02 15:32:16 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here is a pupper approaching maximum borkdrive. Zooming at never before seen speeds. 14/10 paw-inspiring af \n(IG: puffie_the_chow) https://t.co/ghXBIIeQZF,NaN,NaN,NaN,https://twitter.com/dog_rates/status/881536004380872706/video/1,14,10,a,None,None,pupper,None
118,869988702071779329,NaN,NaN,2017-05-31 18:47:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: We only rate dogs. This is quite clearly a smol broken polar bear. We'd appreciate if you only send dogs. Thank you... 12/10…,8.591970e+17,4.196984e+09,2017-05-02 00:04:57 +0000,https://twitter.com/dog_rates/status/859196978902773760/video/1,12,10,quite,None,None,None,None
169,859196978902773760,NaN,NaN,2017-05-02 00:04:57 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",We only rate dogs. This is quite clearly a smol broken polar bear. We'd appreciate if you only send dogs. Thank you... 12/10 https://t.co/g2nSyGenG9,NaN,NaN,NaN,https://twitter.com/dog_rates/status/859196978902773760/video/1,12,10,quite,None,None,None,None
193,855459453768019968,NaN,NaN,2017-04-21 16:33:22 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","Guys, we only rate dogs. This is quite clearly a bulbasaur. Please only send dogs. Thank you... 12/10 human used pet, it's super effective https://t.co/Xc7uj1C64x",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/855459453768019968/photo/1,https://twitter.com/dog_rates/status/855459453768019968/photo/1",12,10,quite,None,None,None,None
335,832645525019123713,NaN,NaN,2017-02-17 17:38:57 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",There's going to be a dog terminal at JFK Airport. This is not a drill. 10/10 \nhttps://t.co/dp5h9bCwU7,NaN,NaN,NaN,http://us.blastingnews.com/news/2017/02/jfk-announces-its-first-ever-ark-oasis-animal-terminal-001480161.html?sbdht=_pM1QUzk3wsdTxcmMoRPV7FWYYlsNKcFRcYSY7OmeHnOXA4NtUM6PLQ2_,10,10,not,None,None,None,None
369,828650029636317184,NaN,NaN,2017-02-06 17:02:17 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","Occasionally, we're sent fantastic stories. This is one of them. 14/10 for Grace https://t.co/bZ4axuH6OK",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/828650029636317184/photo/1,https://twitter.com/dog_rates/status/828650029636317184/photo/1,https://twitter.com/dog_rates/status/828650029636317184/photo/1",14,10,one,None,None,None,None
542,806219024703037440,NaN,NaN,2016-12-06 19:29:28 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",We only rate dogs. Please stop sending in non-canines like this Freudian Poof Lion. This is incredibly frustrating... 11/10 https://t.co/IZidSrBvhi,NaN,NaN,NaN,https://twitter.com/dog_rates/status/806219024703037440/photo/1,11,10,incredibly,None,None,None,None
649,792913359805018113,NaN,NaN,2016-10-31 02:17:31 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here is a perfect example of someone who has their priorities in order. 13/10 for both owner and Forrest https://t.co/LRyMrU7Wfq,NaN,NaN,NaN,"https://twitter.com/dog

All 109 of these look like errors. And I suspect we're just getting started with all the dirty data in this column. And while it would require a lot of effort to scrub this to 100%, I'm not sold that it will yield too many interesting results. Because they are names, there will be many unique values that don't lend themselves to categorical treatment or analysis. Our analyses aren't likely to be impeded by keeping the dogs anonymized.

Now onto the ratings, one of the charms of the twitter account and a interesting avenue for potential analyses. If we're lucky (or stubbornly determined), we'll find reasons to spend more time working on this.

In [27]:
twitter_archive.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7       55 
14      54 
5       37 
6       32 
3       19 
4       17 
1       9  
2       9  
420     2  
0       2  
15      2  
75      2  
80      1  
20      1  
24      1  
26      1  
44      1  
50      1  
60      1  
165     1  
84      1  
88      1  
144     1  
182     1  
143     1  
666     1  
960     1  
1776    1  
17      1  
27      1  
45      1  
99      1  
121     1  
204     1  
Name: rating_numerator, dtype: int64

In [28]:
twitter_archive.rating_denominator.value_counts()

10     2333
11     3   
50     3   
80     2   
20     2   
2      1   
16     1   
40     1   
70     1   
15     1   
90     1   
110    1   
120    1   
130    1   
150    1   
170    1   
7      1   
0      1   
Name: rating_denominator, dtype: int64

As one may recall from seeing WeRateDog tweets, the denominators cluster overwhelmingly around (10), with the numerators pretty tightly centered just above. However, we see plenty of unusual numbers as well - 4-digit numerators and 3-digit denominators, for example. These could be errors produced by the methods used to populate the fields, or some extreme-puppery that may be worth manual examination.

##### Observations

Here are the consolidated notes from the assessment phase.

Quality -

     - The record counts do not match between data sources

    'twitter_archive'
        - "source" column mistypified; should be category. Needs to be parsed to be intelligible
        - "doggo", "floofer", "pupper", "puppo" columns are strings; can be melted into single column
        - Numerators and denominators appear to contain both inconsistent and inaccurate data
        - There appears to be (137) duplicated "expanded_urls" values
        - the timestamp columns are strings
        - contains retweets and replies, ie. non-rating tweets
        - Some columns contain NaNs: 'in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id',
            'retweeted_status_user_id','retweeted_status_timestamp','expanded_urls'
                * The NaNs in the 'in_reply...' and 'retweet...' columns will be helpful for identifying non-ratings tweets
                * The explicit requirement is for original rating tweets with images - so missing urls may be dropped as 
                   well
        - There are (109) values in the 'name' column that are all lowercase, which appear to be dirty data (as proper 
        nouns, these should have at least one capital)
        - tweet_id 	832645525019123713 did not contain an actual dog rating

    
    'twitter_api_data'
        - columns imported as strings instead of integers
    
    'image_predictions'
        - Column headings are not intelligible
        - There appears to be duplicated "jpg_url" fields

Tidiness -

    - Dog 'stage' shouldn't be spread across multiple columns
    - All (3) sources can be merged

### Data Cleaning
#### Preparation - Copying Our Dataframes

###### Define:
    To retain the integrity of our original data and simplfy rolling-back changes if we produce errors while cleaning our 
    data, we will make copies of all our dataframes prior to cleaning. We will do this by creating new dataframes from our 
    old ones with the copy() method
    
###### Code:

In [29]:
twitter_archive_clean = twitter_archive.copy()
twitter_api_data_clean = twitter_api_data.copy()
image_predictions_clean = image_predictions.copy()

##### Test

In [30]:
print ("Records in Twitter Archive: ",len(twitter_archive_clean))
print ("Records in Twitter API Data: ",len(twitter_api_data_clean))
print ("Records in Image Predictions: ",len(image_predictions_clean))

Records in Twitter Archive:  2356
Records in Twitter API Data:  2340
Records in Image Predictions:  2075


### Completeness
A first pass will be made toward completeness, to ease further cleanup efforts. Initial observations showed that the dataframes did not contain an equal number of records, and I doubt there's full overlap with the shortest table. Likewise, some records must be dropped from the 'twitter_archive' and 'twitter_api_data' dataframes in order to meet the requirement to filter for only original rating tweets.

##### Drop Non-Rating Tweets
We're going to remove all records with values in either the "in_reply..." or "retweeted..." columns, indicating that the tweet is not an original tweet. Let's first identify and quantify them, then remove them.

###### Define:
    We will use the query method to select any rows where the 'in_reply_to_status_id' or 'retweet_status_id' do not 
    contain a aN value. First we will print the count of NaNs and their proportion of the record set, so that we can 
    later test that all have been dropped.
    
    Next we will redefine the cleaned dataframe with a query using the inverse logic: only where the reply and 
    retweet columns are NaN. Lastly, we will take the opportunity to manually drop the tweet about JFK airport
    identified above.

##### Code:

In [31]:
retweets_and_replies = twitter_archive_clean.query('in_reply_to_status_id!="NaN" | retweeted_status_id!="NaN"')
print(len(retweets_and_replies),' records are retweets or relpies')
print((len(retweets_and_replies)/len(twitter_archive_clean)*100),' % of the twitter archive')

259  records are retweets or relpies
10.99320882852292  % of the twitter archive


In [32]:
twitter_archive_clean = twitter_archive_clean.query('in_reply_to_status_id=="NaN" and retweeted_status_id=="NaN"')
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean.tweet_id!=832645525019123713]

##### Test
We should expect (2356) - (259) - (1) = (2096)

In [33]:
len(twitter_archive_clean)

2096

##### Drop Tweets Without Images
Next, we clean out all the tweets without images. If the tweet has no URL, there's no image in the tweet. So we can look for all records missing values in the 'expanded_urls' column.

###### Define:
    We will use the isnull() method to select and remove records where the 'expanded_urls' column contains a NaN. 
    
##### Code:

In [34]:
no_url = twitter_archive_clean.expanded_urls.isnull()
print(len(twitter_archive_clean[no_url]))
twitter_archive_clean[no_url]

3


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
375,828361771580813312,NaN,NaN,2017-02-05 21:56:51 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",Beebop and Doobert should start a band 12/10 would listen,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
707,785515384317313025,NaN,NaN,2016-10-10 16:20:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","Today, 10/10, should be National Dog Rates Day",NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
1445,696518437233913856,NaN,NaN,2016-02-08 02:18:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Oh my god 10/10 for every little hot dog pupper,NaN,NaN,NaN,NaN,10,10,None,None,None,pupper,None


In [35]:
has_url = twitter_archive_clean.expanded_urls.notnull()
twitter_archive_clean = twitter_archive_clean[has_url]

##### Test
We should expect (2090) - (3) = (2093) records, and no NaNs in the column.

In [36]:
print(len(twitter_archive_clean))
twitter_archive_clean.isnull().sum()/len(twitter_archive_clean)

2093


tweet_id                      0.0
in_reply_to_status_id         1.0
in_reply_to_user_id           1.0
timestamp                     0.0
source                        0.0
text                          0.0
retweeted_status_id           1.0
retweeted_status_user_id      1.0
retweeted_status_timestamp    1.0
expanded_urls                 0.0
rating_numerator              0.0
rating_denominator            0.0
name                          0.0
doggo                         0.0
floofer                       0.0
pupper                        0.0
puppo                         0.0
dtype: float64

##### Drop Non-Original Rating Tweets (Duplicated Images)
It's possible that there is still dirty data in the column, even though there are no NaNs. We can check and clean this programmatically as well.

##### Define:
    Use the len function and drop_duplicates() method to identify if there are duplicated values in the 'expanded_urls' 
    column. Print them for review. They will be dropped if they do not meet the constraint of an "original ratings tweet".

##### Code:

In [37]:
print("Potential Dupes - twitter_archive.expanded_urls - ",len(twitter_archive_clean.expanded_urls)-len(twitter_archive_clean.expanded_urls.drop_duplicates()))

Potential Dupes - twitter_archive.expanded_urls -  1


In [38]:
twitter_archive_clean[twitter_archive_clean.duplicated(['expanded_urls'],keep=False)]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
657,791774931465953280,NaN,NaN,2016-10-27 22:53:48 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine - Make a Scene</a>",Vine will be deeply missed. This was by far my favorite one. 14/10 https://t.co/roqIxCvEB3,NaN,NaN,NaN,https://vine.co/v/ea0OwvPTx9l,14,10,None,None,None,None,None
2212,668587383441514497,NaN,NaN,2015-11-23 00:30:28 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine - Make a Scene</a>",Never forget this vine. You will not stop watching for at least 15 minutes. This is the second coveted.. 13/10 https://t.co/roqIxCvEB3,NaN,NaN,NaN,https://vine.co/v/ea0OwvPTx9l,13,10,the,None,None,None,None


It looks like most of the duplicates were dropped with all of the retweets and replies. There was only one real duplicate, which was - in fact - a repost. This is exusable, as WeRateDogs was honoring the decommissioning of Vine by upgrading their prior review for that dog by (1) point. As such, we decide to drop a duplicate and retain the newer, amended rating tweet.

In [39]:
twitter_archive_clean.drop_duplicates(subset='expanded_urls',keep='first',inplace=True)

##### Test
We should expect (2093) - (1) = (2092) records

In [40]:
len(twitter_archive_clean)

2092

### Tidiness

#### Melt the 'Dog Stage' Columns

##### Define:
    Udacity requires that the 'doggo', 'floofer', 'pupper', and 'puppo' columns be merged for tidiness. This will 
    be accomplished by passing lists of columns to the melt function, then dropping the duplicates that will be produced.

##### Code:

In [41]:
dog_stage_cols = ['doggo','floofer','pupper','puppo']
all_other_cols = [x for x in twitter_archive_clean.columns.tolist() if x not in dog_stage_cols]
twitter_archive_clean = pd.melt(twitter_archive_clean,id_vars=all_other_cols,value_vars=dog_stage_cols,var_name='stage',value_name='dog_stage')
twitter_archive_clean = twitter_archive_clean.sort_values('dog_stage').drop_duplicates('tweet_id',keep='last')
twitter_archive_clean.drop('stage',axis=1,inplace=True)

##### Test:
There should still be (2092) records after deleting the duplicate tweet_ids, and the 'dog_stage' column should now contain all our values.

In [42]:
print(len(twitter_archive_clean))
twitter_archive_clean.dog_stage.value_counts()

2092


None       1757
pupper     229 
doggo      72  
puppo      24  
floofer    10  
Name: dog_stage, dtype: int64

In [43]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2092 entries, 2016 to 6387
Data columns (total 14 columns):
tweet_id                      2092 non-null int64
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     2092 non-null object
source                        2092 non-null object
text                          2092 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 2092 non-null object
rating_numerator              2092 non-null int64
rating_denominator            2092 non-null int64
name                          2092 non-null object
dog_stage                     2092 non-null object
dtypes: float64(4), int64(3), object(7)
memory usage: 245.2+ KB


####  Merge the (3) Data Sources
Now that we're increasingly confident that our archive data is more complete and representative of our desired dataset, we can consider issues about tidiness. Furthermore, to progress on completeness, we need to understand where inconsistent record counts will result in records that do not join - and thereby produce more NaNs across our finished table.

##### Define:
    Set the indexes of the 'twitter_archive_clean', 'twitter_api_data_clean' and 'image_predictions_clean
    dataframes to their tweet_ids, and join them. Evaluate for NaNs.
    
##### Code:

In [44]:
twitter_archive_master = twitter_archive_clean.set_index('tweet_id').join(twitter_api_data_clean.set_index('tweet_id')).join(image_predictions_clean.set_index('tweet_id'))

##### Test:

In [45]:
print(len(twitter_archive_master))
print(twitter_archive_master.isnull().sum()/len(twitter_archive_master))

2092
in_reply_to_status_id         1.000000
in_reply_to_user_id           1.000000
timestamp                     0.000000
source                        0.000000
text                          0.000000
retweeted_status_id           1.000000
retweeted_status_user_id      1.000000
retweeted_status_timestamp    1.000000
expanded_urls                 0.000000
rating_numerator              0.000000
rating_denominator            0.000000
name                          0.000000
dog_stage                     0.000000
favorite_count                0.000956
retweet_count                 0.000956
jpg_url                       0.057839
img_num                       0.057839
p1                            0.057839
p1_conf                       0.057839
p1_dog                        0.057839
p2                            0.057839
p2_conf                       0.057839
p2_dog                        0.057839
p3                            0.057839
p3_conf                       0.057839
p3_dog              

In [46]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2092 entries, 667405339315146752 to 867421006826221569
Data columns (total 26 columns):
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     2092 non-null object
source                        2092 non-null object
text                          2092 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 2092 non-null object
rating_numerator              2092 non-null int64
rating_denominator            2092 non-null int64
name                          2092 non-null object
dog_stage                     2092 non-null object
favorite_count                2090 non-null object
retweet_count                 2090 non-null object
jpg_url                       1971 non-null object
img_num                       1971 non-null float64
p1     

Joining our tables has improved tidiness, but we now need to iterate back into completeness, as there are record breaks between our data sources (evidenced by the NaNs in columns originall from the API and prediction data). We only have full data for (1969) records between all of the data sources.

##### Define:
    Identify, select, and print the NaNs originating from the API and prediction data. If appropriate, drop these 
    from the dataset. 
    
##### Code:

In [47]:
null_favorites = twitter_archive_master.favorite_count.isnull()
twitter_archive_master[null_favorites]

,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
tweet_id,,,,,,,,,,,,,,,,,,,,,
680055455951884288,NaN,NaN,2015-12-24 16:00:30 +0000,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>","Meet Sammy. At first I was like ""that's a snowflake. we only rate dogs,"" but he would've melted by now, so 10/10 https://t.co/MQfPK4zwuh",NaN,NaN,NaN,https://twitter.com/dog_rates/status/680055455951884288/photo/1,10,...,1.0,Samoyed,0.995466,True,Great_Pyrenees,0.001834,True,Pomeranian,0.000667,True
754011816964026368,NaN,NaN,2016-07-15 17:56:40 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Charlie. He pouts until he gets to go on the swing. 12/10 manipulative af https://t.co/ilwQqWFKCh,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/754011816964026368/photo/1,https://twitter.com/dog_rates/status/754011816964026368/photo/1",12,...,1.0,French_bulldog,0.600985,True,Boston_bull,0.273176,True,boxer,0.056772,True


After a manual investigation, both of these tweets have been deleted. They will be dropped from our master table.

In [48]:
not_deleted_tweets = twitter_archive_master.favorite_count.notnull()
twitter_archive_master = twitter_archive_master[not_deleted_tweets]

In [49]:
twitter_archive_master[twitter_archive_master.p1.isnull()]

,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
tweet_id,,,,,,,,,,,,,,,,,,,,,
875097192612077568,NaN,NaN,2017-06-14 21:06:43 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",You'll get your package when that precious man is done appreciating the pups. 13/10 for everyone https://t.co/PFp4MghzBW,NaN,NaN,NaN,https://twitter.com/drboondoc/status/874413398133547008,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
876537666061221889,NaN,NaN,2017-06-18 20:30:39 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",I can say with the pupmost confidence that the doggos who assisted with this search are heroic as h*ck. 14/10 for all https://t.co/8yoc1CNTsu,NaN,NaN,NaN,https://twitter.com/mpstowerham/status/876162994446753793,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
878604707211726852,NaN,NaN,2017-06-24 13:24:20 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Martha is stunning how h*ckin dare you. 13/10 https://t.co/9uABQXgjwa,NaN,NaN,NaN,https://twitter.com/bbcworld/status/878599868507402241,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
885518971528720385,NaN,NaN,2017-07-13 15:19:09 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",I have a new hero and his name is Howard. 14/10 https://t.co/gzLHboL7Sk,NaN,NaN,NaN,https://twitter.com/4bonds2carbon/status/885517367337512960,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
884247878851493888,NaN,NaN,2017-07-10 03:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",OMG HE DIDN'T MEAN TO HE WAS JUST TRYING A LITTLE BARKOUR HE'S SUPER SORRY 13/10 WOULD FORGIVE IMMEDIATE https://t.co/uF3pQ8Wubj,NaN,NaN,NaN,https://twitter.com/kaijohnson_19/status/883965650754039809,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
690989312272396288,NaN,NaN,2016-01-23 20:07:44 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine - Make a Scene</a>",We've got a doggy down. Requesting backup. 12/10 for both. Please enjoy https://t.co/pmarb2dG0e,NaN,NaN,NaN,https://vine.co/v/iOZKZEU2nHq,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
691793053716221953,NaN,NaN,2016-01-26 01:21:31 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine - Make a Scene</a>",We usually don't rate penguins but this one is in need of a confidence boost after that slide. 10/10 https://t.co/qnMJHBxPuo,NaN,NaN,NaN,https://vine.co/v/OTTVAKw6YlW,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
692041934689402880,NaN,NaN,2016-01-26 17:50:29 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine - Make a Scene</a>",This is Teddy. His head is too heavy. 13/10 (vid by @jooanrim) https://t.co/sRUpRpGZ3y,NaN,NaN,NaN,https://vine.co/v/iiI3wmqXYmA,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
690348396616552449,NaN,NaN,2016-01-22 01:40:58 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine - Make a Scene</a>",This is Oddie. He's trying to communicate. 12/10 very solid effort (vid by @kaleseyy) https://t.co/JjxriLqZOL,NaN,NaN,NaN,https://vine.co/v/iejBWerY9X2,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


These all appear to be legitimate, original-rating tweets. There are two approaches here:
    1) Drop them for the missing prediction data
    2) Retain them, and drop them as-needed for analysis using the prediction data.

Because Udacity has emphasized an intended output from the project with a single master dataframe with 'complete' data, these will be dropped. 

In [50]:
has_predictions = twitter_archive_master[twitter_archive_master.p1.notnull()]
twitter_archive_master = has_predictions

##### Test:
We expect the dataset to be reduced to (1969) records

In [51]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1969 entries, 667405339315146752 to 867421006826221569
Data columns (total 26 columns):
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     1969 non-null object
source                        1969 non-null object
text                          1969 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 1969 non-null object
rating_numerator              1969 non-null int64
rating_denominator            1969 non-null int64
name                          1969 non-null object
dog_stage                     1969 non-null object
favorite_count                1969 non-null object
retweet_count                 1969 non-null object
jpg_url                       1969 non-null object
img_num                       1969 non-null float64
p1     

## Remaining Quality Issues

#### Changing Data Types
Now that we've addressed the more structural issue with the data, we perform some quick housekeeping to correct datatypes. This will facilitate all of our analyses operations later on.

##### Define:
    From the 'twitter_archive_master' dataframe the 'favorite_count' and 'retweet_count' will be converted to integers
    using the "astype" method

##### Code:

In [52]:
#change the favorite_count and retweet_count to ints
twitter_archive_master['favorite_count'] = twitter_archive_master['favorite_count'].astype(str).astype(int)
twitter_archive_master['retweet_count'] = twitter_archive_master['retweet_count'].astype(str).astype(int)

##### Test:
We expect to see these columns as datatype 'int64'

In [53]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1969 entries, 667405339315146752 to 867421006826221569
Data columns (total 26 columns):
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     1969 non-null object
source                        1969 non-null object
text                          1969 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 1969 non-null object
rating_numerator              1969 non-null int64
rating_denominator            1969 non-null int64
name                          1969 non-null object
dog_stage                     1969 non-null object
favorite_count                1969 non-null int64
retweet_count                 1969 non-null int64
jpg_url                       1969 non-null object
img_num                       1969 non-null float64
p1       

##### Define:
    From the 'twitter_archive_master' dataframe the 'timestamp' and 'retweeted_status_timestamp' fields will be converted
    to datetime by passing a datetime object
    
##### Code:

In [54]:
datetimes_to_convert = ['timestamp','retweeted_status_timestamp']
for col in datetimes_to_convert:
    twitter_archive_master[col]= pd.to_datetime(twitter_archive_master[col])

##### Test:
We expect to see these columns as datatype 'datetime64'

In [55]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1969 entries, 667405339315146752 to 867421006826221569
Data columns (total 26 columns):
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     1969 non-null datetime64[ns]
source                        1969 non-null object
text                          1969 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null datetime64[ns]
expanded_urls                 1969 non-null object
rating_numerator              1969 non-null int64
rating_denominator            1969 non-null int64
name                          1969 non-null object
dog_stage                     1969 non-null object
favorite_count                1969 non-null int64
retweet_count                 1969 non-null int64
jpg_url                       1969 non-null object
img_num                       1969 non-null f

#### Parse the 'source' Column

###### Define:
    From the 'twitter_archive_master' dataframe the 'source' column will be parsed using the split() method to remove HTML,
    and then converted to a category by passing a Categorical object

##### Code:

In [56]:
# Initial counts for sanity check later
twitter_archive_master.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     1931
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                     28  
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>    10  
Name: source, dtype: int64

Next up, we can clean-up all the HTML from the 'source' column, and convert it to a category. This may come in handy for analyses and visualizations.

In [57]:
source_parse_left = twitter_archive_master.source.str.split('>',expand=True)
source_parse_right = source_parse_left[1].str.split("<", expand=True)
twitter_archive_master['source']=source_parse_right[0]
twitter_archive_master['source'] = pd.Categorical(twitter_archive_master['source'])

##### Test:
We expect to see these values without HTML, but the same counts as before. The column datatype should now read 'category'

In [58]:
print(twitter_archive_master.source.value_counts())
twitter_archive_master.info()

Twitter for iPhone    1931
Twitter Web Client    28  
TweetDeck             10  
Name: source, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1969 entries, 667405339315146752 to 867421006826221569
Data columns (total 26 columns):
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     1969 non-null datetime64[ns]
source                        1969 non-null category
text                          1969 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null datetime64[ns]
expanded_urls                 1969 non-null object
rating_numerator              1969 non-null int64
rating_denominator            1969 non-null int64
name                          1969 non-null object
dog_stage                     1969 non-null object
favorite_count                1969 non-null int64
retweet_count                 1969 n

##### Improve Rating Accuracy
While the project's instructions say that extreme values in the ratings columns are valid (ie. it is allowed in the column), they may also signal accuracy issues. It's worth exploring, especially since the quantititative values make the problem more tractable and perhaps easier to clean than text fields.

Iterating back in the data wrangingly process, we will do further assessment by examining the extreme values with a mix of programmatic and manual analysis. To accomplish this, we will:

    - Create a 'dog score' by dividing the numerator by the denominator. Because some extreme values in both the numerator 
    and denominator may relate to eachother, this should reduce noise if both extreme values are proportioned and 
    in-line with the other ratings
    - We will set thresholds for the top and bottom 1% of dog scores, and print the URLs for scores beyond these thresholds
    - We will manually review these tweets to determine if there errors in capturing the data, or if the scores are legitimate

In [59]:
# Calculating the dog score and printing out the value counts
twitter_archive_master['dog_score'] = twitter_archive_master['rating_numerator'] / twitter_archive_master['rating_denominator']
twitter_archive_master.dog_score.value_counts() 

1.200000      449
1.000000      418
1.100000      397
1.300000      253
0.900000      150
0.800000      95 
0.700000      51 
0.500000      34 
1.400000      33 
0.600000      32 
0.300000      19 
0.400000      15 
0.200000      10 
0.100000      4  
177.600000    1  
2.600000      1  
3.428571      1  
0.636364      1  
0.818182      1  
0.000000      1  
7.500000      1  
42.000000     1  
2.700000      1  
Name: dog_score, dtype: int64

In [60]:
# Setting the boundaries. Looking at the 1% tails is arbitrary, and these can be adjusted again if I iterate back in this line of inquiry 
upper_bound = twitter_archive_master['dog_score'].quantile(0.99)
lower_bound = twitter_archive_master['dog_score'].quantile(0.01)
print(upper_bound, lower_bound)

1.4 0.3


In [61]:
# Here we look for all tweets where the score exceeds 99% of other scores. 
# Because I want to review the manually, I only print the key informaiton needed to do this: the ID, and the URL
print("Number of Dog Scores Above the 99% Percentile : ",len(twitter_archive_master[twitter_archive_master['dog_score']>upper_bound]))
above_upper_bound = twitter_archive_master[twitter_archive_master['dog_score']>upper_bound]
above_upper_bound['expanded_urls']

Number of Dog Scores Above the 99% Percentile :  6


tweet_id
670842764863651840    https://twitter.com/dog_rates/status/670842764863651840/photo/1                                    
680494726643068929    https://twitter.com/dog_rates/status/680494726643068929/photo/1                                    
786709082849828864    https://twitter.com/dog_rates/status/786709082849828864/photo/1                                    
810984652412424192    https://www.gofundme.com/sams-smile,https://twitter.com/dog_rates/status/810984652412424192/photo/1
749981277374128128    https://twitter.com/dog_rates/status/749981277374128128/photo/1                                    
778027034220126208    https://twitter.com/dog_rates/status/778027034220126208/photo/1                                    
Name: expanded_urls, dtype: object

In [62]:
# Here we look for all tweets where the score below 1% of other scores. 
# Because I want to review the manually, I only print the key informaiton needed to do this: the ID, and the URL
print("Number of Dog Scores Below the 1% Percentile : ",len(twitter_archive_master[twitter_archive_master['dog_score']<lower_bound]))
below_lower_bound = twitter_archive_master[twitter_archive_master['dog_score']<lower_bound]
below_lower_bound['expanded_urls']

Number of Dog Scores Below the 1% Percentile :  15


tweet_id
667549055577362432    https://twitter.com/dog_rates/status/667549055577362432/photo/1                                                                                                                                
666786068205871104    https://twitter.com/dog_rates/status/666786068205871104/photo/1                                                                                                                                
667878741721415682    https://twitter.com/dog_rates/status/667878741721415682/photo/1                                                                                                                                
668142349051129856    https://twitter.com/dog_rates/status/668142349051129856/photo/1                                                                                                                                
666104133288665088    https://twitter.com/dog_rates/status/666104133288665088/photo/1                                                  

We make more observations based on manual assessment of the extreme scores.

    For scores above the threshold, there appear to be three causes:
        1) (3) Errors in programmatically extracting values with decimals, eg. '9.75' captured as '75'
            Tweet IDs:
                - 786709082849828864 9.75/10
                - 778027034220126208 11.27/10
                - 680494726643068929 11.26/10
        2) (2) Jokes that play-on the rating system, based on the contents of the photo
            Tweet IDs:
                - 670842764863651840 420/10 - Snoop Dogg 
                - 749981277374128128 1776/10 - Atticus, dressed up in stars & stripes
        3) Promotion of a GoFundMe request, with the description 'smiling 24/7' captured as a rating
            Tweet ID: 810984652412424192
      
    For scores below the threshold, there appears to be three cause:
        1) Jokes about animals that are not dogs, who score low for missing critical dog traits
        2) Jokes about dogs who are doing something naughty or otherwise not meeting their potential
        3) (2) data capture errors
            Tweet IDs:
                - 722974582966214656 4/20 date captured before 13/10 score
                - 667878741721415682 2&10 / 10 joke about steering wheel positions
        
Within the upper bound, (3) records are clearly inaccurate due to programmatic extraction, and can be amended manually. The GoFundMe solicitation does not meet the requirement for an original ratings tweet, and therefore should be dropped. The remaining (2) are inconsistent, as they are scored according to arbitrary metrics that differ from the others dogs. However, because the entire scoring system is somewhat arbitrary, fit the validation rule, and appear accurate.

For scores below the lower bound, there is a mix of legitimately low-scoring dogs as well as non-dogs, and (2) data capture errors. The programmatic approach appears to have accurately captured most of the scores from the tweets, with minimal manual amendment required. While there are consistency issues, ie. some tweets are not rating dogs, they appear accurate and valid, and we will therefore leave them in the data (and use these characteristics in our analyses later, since the prediction data attempts to catch this).

For the data capture errors - these will be manually updated with the correct score from the tweet. The "2&10/10" joke will be interpreted literally as 2 + 10 / 10, for as score of 12/10. To perform these updates, I will make dictionaries with the tweet_id and the corrected numerator and denominators, use them to update the records, and print one of them to confirm.

#### Cleaning
##### Define:
    We will manually drop the GoFundMe tweet and define dictionaries to correct the numerators and denominators, which 
    will be passed to the update() method to amend our data.
    
    Note: This approach is not scalable or robust as additional programmatic options, but it is perfectly reasonable 
    since we've identified fewer than (10) records requiring amendment. A programmatic approach to 
    parse all numerators and denominators with decimals, for example, would not catch all of the breaks identified 
    (eg. '2&10') - thus requiring the definition, coding, and testing of additional programmatic methods. Alternatively, 
    manual updates here are both precise and expedient and appropriately sized for the scale of the problem.

In [63]:
# The GoFundMe tweet was benevolent, but not within our constraints
twitter_archive_master.drop(810984652412424192,axis=0,inplace=True)

In [64]:
# We create our dictionaries. The tweet_ids are now indexes in the dataframes instead of columns
update_numerators = {786709082849828864:9.75,778027034220126208:11.27,680494726643068929:11.26,722974582966214656:13,667878741721415682:12}
update_denominators =  {786709082849828864:10,778027034220126208:10,680494726643068929:10,722974582966214656:10,667878741721415682:10}

# We use the update method on the corresponding column, passing the tweet_ids as index locations for the update 
twitter_archive_master['rating_numerator'].update(pd.Series(update_numerators))
twitter_archive_master['rating_denominator'].update(pd.Series(update_denominators))

##### Test:
We print the dataframe length, expecting to see (1968) records. We then print one of the amended records and expect to see (12) in the numerator and (10) in the denominator. Finally, we recalculate the dog scores and re-print the upper threshold, expecting to see (2) legitimate, accurate outliers.

In [65]:
len(twitter_archive_master)

1968

In [66]:
twitter_archive_master.loc[667878741721415682]

in_reply_to_status_id         NaN                                                                                                                                        
in_reply_to_user_id           NaN                                                                                                                                        
timestamp                     2015-11-21 01:34:35                                                                                                                        
source                        Twitter for iPhone                                                                                                                         
text                          This is Tedrick. He lives on the edge. Needs someone to hit the gas tho. Other than that he's a baller. 10&amp;2/10 https://t.co/LvP1TTYSCN
retweeted_status_id           NaN                                                                                                                     

In [67]:
# Recalculating dog scores and checking the top percentile again to confirm our changes.
twitter_archive_master['dog_score'] = twitter_archive_master['rating_numerator'] / twitter_archive_master['rating_denominator'] 
upper_bound = twitter_archive_master['dog_score'].quantile(0.99)
lower_bound = twitter_archive_master['dog_score'].quantile(0.01)
print("Number of Dog Scores Above the 99% Percentile : ",len(twitter_archive_master[twitter_archive_master['dog_score']>upper_bound]))
above_upper_bound = twitter_archive_master[twitter_archive_master['dog_score']>upper_bound]
above_upper_bound['expanded_urls']

Number of Dog Scores Above the 99% Percentile :  2


tweet_id
670842764863651840    https://twitter.com/dog_rates/status/670842764863651840/photo/1
749981277374128128    https://twitter.com/dog_rates/status/749981277374128128/photo/1
Name: expanded_urls, dtype: object

In [68]:
twitter_archive_master.describe()

,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator,favorite_count,retweet_count,img_num,p1_conf,p2_conf,p3_conf,dog_score
count,0.0,0.0,0.0,0.0,1968.000000,1968.000000,1968.000000,1968.000000,1968.000000,1968.000000,1.968000e+03,1.968000e+03,1968.000000
mean,NaN,NaN,NaN,NaN,12.179512,10.474593,8743.462398,2652.873476,1.202236,0.594210,1.346318e-01,6.021497e-02,1.164625
std,NaN,NaN,NaN,NaN,41.637331,6.854680,12806.606532,4722.910250,0.559391,0.272112,1.010130e-01,5.095678e-02,4.090556
min,NaN,NaN,NaN,NaN,0.000000,2.000000,78.000000,11.000000,1.000000,0.044333,1.011300e-08,1.740170e-10,0.000000
25%,NaN,NaN,NaN,NaN,10.000000,10.000000,1881.500000,590.750000,1.000000,0.362903,5.352722e-02,1.609288e-02,1.000000
50%,NaN,NaN,NaN,NaN,11.000000,10.000000,3942.000000,1275.500000,1.000000,0.587440,1.173995e-01,4.945765e-02,1.100000
75%,NaN,NaN,NaN,NaN,12.000000,10.000000,10899.750000,3029.750000,1.000000,0.847062,1.955618e-01,9.157912e-02,1.200000
max,NaN,NaN,NaN,NaN,1776.000000,170.000000,163518.000000,83141.000000,4.000000,1.000000,4.880140e-01,2.734190e-01,177.600000


### Cleaning Summary
Thus far two tidiness issues have been addressed:
    1) Melting the 'doggo', 'floofer', 'puppo', and 'pupper' columns into a single 'dog_stage' column
    2) The Twitter archive, API, and prediction data sources have been merged

Additionally, nine quality issues have been addressed:
    - Only original rating tweets remain, arrived at by:
        1) Dropping retweets and replies
        2) Dropping tweets without images
        4) Dropping tweets with duplicate images
        5) Dropping (1) tweet containing a duplicated image URL
        6) Dropping (1) tweet containing a GoFundMe solicitation, and no rating
    - Converting datatypes where required for utility and consistency
        7) Parsing out the HTML from the 'source' column and converting it to a categorical datatype
        8) Converting the 'timestamp' and 'retweet_timestamp' columns to datetime datatypes
        9) 'twitter_api_data' columns read as strings have been converted to integers
    - Cleansed dirty values from 'rating_numerator' and 'rating_denominator' columns through the identification of
        both extreme and mis-proportioned values, manually identifed errors in programmatic extraction
        10) Applied corrected numerator and denominator values from each tweet using a dictionary of tweet_ids and values
 
From the original assessment observations, the below remain unaddressed:

    1) There known data quality isuses in the 'name' column. However, we will not address this, as perfect data quality here is 
     not required by the project, and the utility of sanitizing this is likely not worth the effort. Effort here does not seem
     fruitful toward improving completeness or tidiness, and anonymization of the dogs does not materially impact our analyses.
         

#### Save the Files with Required Names

In [69]:
twitter_archive_master.to_csv('twitter_archive_master.csv',index=False)

## Resources Used

 - Udacity DAND classroom materials
 - Stack Overflow : https://stackoverflow.com/
 - Stack Abuse : https://stackabuse.com/
 - Pandas documentation : https://pandas.pydata.org/pandas-docs/stable/index.html
 - Tweepy documentation : http://docs.tweepy.org/en/v3.5.0/
 - JSON documentation : https://docs.python.org/3/library/json.html
 - cran-r.project.org, 'Tidy Data' : https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html
 - docs.python.org, request documentation : http://docs.python-requests.org/en/master/user/quickstart/
 - docs.python.org, os documentation : https://docs.python.org/3/library/os.path.html